In [ ]:
import json

from collections import Counter
from copy import deepcopy
from dateutil import parser
from datetime import timezone
from pprint import pprint

import ipywidgets as ipyw
import traitlets as trt
from IPython.display import display

import rdflib as rdf
import networkx as nx

import requests

import ipyelk
import ipyelk.nx
from ipyelk.diagram.elk_model import ElkLabel

In [ ]:
try:
    import sysml_v2_api_client as sysml2
    from sysml_v2_api_client.rest import ApiException
except ImportError:
    !pip install git+https://github.com/Systems-Modeling/SysML-v2-API-Python-Client.git
    print(
        "Had to install the SysML v2 Python API Client.\n"
        "Restart the kernel and run the notebook again..."
    )

In [ ]:
class SysML2LabeledPropertyGraph(trt.HasTraits):

    graph = trt.Instance(nx.MultiDiGraph, args=tuple())
    
    def __repr__(self):
        return (
            "<SysML v2 LPG: "
            f"{len(self.graph.nodes):,d} nodes, "
            f"{len(self.graph.edges):,d} edges"
            ">"
        )
    
    def _update(self, client, merge=False):
        if not merge:
            old_graph = self.graph
            self.graph = nx.MultiDiGraph()
            del old_graph
            
        elements_by_id = client.elements_by_id

        relationship_element_ids = {
            element_id
            for element_id, element in elements_by_id.items()
            if 'relatedElement' in element
        }
        non_relationship_element_ids = set(
            elements_by_id
        ).difference(relationship_element_ids)
        
        self.graph.add_nodes_from(
            {
                element_id: elements_by_id[element_id]
                for element_id in non_relationship_element_ids
            }.items()
        )
        relationships = [
            elements_by_id[id_]
            for id_ in relationship_element_ids
        ]
        self.graph.add_edges_from([
            [
                relation["relatedElement"][0]["@id"],  # source node (str id)
                relation["relatedElement"][1]["@id"],  # target node (str id)
                relation["@type"],                     # edge type (str name)
                relation,                              # edge data (dict)
            ]
            for relation in relationships
        ])
        
    def subgraph(
        self,
        *,
        add_ipyelk_data: bool = True,
        edges: (list, tuple) = None,
        edge_types: (list, tuple, str) = None,
    ):
        graph = self.graph
        subgraph = type(graph)()

        edges = edges or []

        edge_types = edge_types or []
        if isinstance(edge_types, str):
            edge_types = [edge_types]

        if edge_types:
            edges += [
                (source, target, data)
                for (source, target, type_), data in graph.edges.items()
                if type_ in edge_types
            ]

        if not edges:
            print(f"Could not find any edges of type: '{edge_types}'!")
            return subgraph

        nodes = {
            node_id: graph.nodes[node_id]
            for node_id in sum([
                [source, target]
                for (source, target, data) in edges
            ], [])  # sum(a_list, []) flattens a_list
        }

        for id_, node_data in nodes.items():
            node_data["id"] = id_
            type_label = [ElkLabel(
                id=f"""type_label_for_{id_}""",
                text=f"""«{node_data["@type"]}»""",
                properties={
                    "cssClasses": "node_type_label",
                },
            )] if "@type" in node_data else []
            node_data["labels"] = type_label + [node_data["name"]]

        subgraph.add_nodes_from(nodes.items())
        subgraph.add_edges_from(edges)

        return subgraph
    
    def make_diagram(self, graph: nx.Graph) -> ipyw.HBox:
        layouts = ipyelk.nx.XELKTypedLayout()

        elk_diagram = ipyelk.Elk(
            transformer=ipyelk.nx.XELK(
                source=(graph, None),
                label_key="labels",
                layouts=layouts.value,
            ),
            style={
                " text.elklabel.node_type_label": {
                    "font-style": "italic",
                }
            },
        )

        def _element_type_opt_change(change):
            elk_diagram.transformer.layouts = layouts.value
            elk_diagram.refresh()

        layouts.observe(_element_type_opt_change, "value")
        elk_diagram.layout.flex = "1"
        
        # Make the direction and label placement look better...
        self.set_layout_option(layouts, "Parents", "Direction", "UP")
        self.set_layout_option(layouts, "Label", "Node Label Placement", "H_CENTER V_TOP INSIDE")

        return ipyw.HBox([elk_diagram, layouts], layout=dict(height="60vh"))
    
    @staticmethod
    def set_layout_option(widget, category: str, option: str, value):
        """Set a layout option"""
        category_idxs = [
            int(idx)
            for idx, name in widget._titles.items()
            if name == category
        ]
        if len(category_idxs) != 1:
            raise ValueError(f"Found {len(category_idxs)} entries for '{category}'!")
        category_widget = widget.children[category_idxs[0]]

        option_idxs = [
            int(idx)
            for idx, name in category_widget._titles.items()
            if name == option
        ]
        if len(option_idxs) != 1:
            raise ValueError(f"Found {len(option_idxs)} entries for '{option}' under '{category}'!")

        category_widget.children[option_idxs[0]].value = value

In [ ]:
class SysML2RDFGraph(trt.HasTraits):

    _cached_contexts: dict = trt.Instance(dict, args=tuple())
    graph: rdf.Graph = trt.Instance(rdf.Graph, args=tuple())

    def import_context(self, jsonld_item: dict) -> dict:
        jsonld_item = deepcopy(jsonld_item)
        context_url = jsonld_item.get("@context", {}).get("@import", None)
        if not context_url:
            return jsonld_item
        if context_url not in self._cached_contexts:
            response = requests.get(context_url)
            if not response.ok:
                raise requests.HTTPError(response.reason)
            data = response.json()
            if "@context" not in data:
                raise ValueError(
                    f"Download context does not have a @context key: {list(data.keys())}"
                )
            self._cached_contexts[context_url] = data["@context"]
        jsonld_item["@context"].update(self._cached_contexts[context_url])
        return jsonld_item
    
    def __repr__(self):
        return (
            "<SysML v2 RDF Graph: "
            f"{len(self.graph):,d} triples, "
            f"{len(set(self.graph.predicates())):,d} unique predicates"
            ">"
        )

    def _update(self, client, merge=False):
        if not merge:
            old_graph = self.graph
            self.graph = rdf.Graph()
            del old_graph

        elements = [
            self.import_context(element)
            for element in client.elements_by_id.values()
        ]
        self.graph.parse(
            data=json.dumps(elements),
            format="application/ld+json",
        )

In [ ]:
class SysML2Client(ipyw.VBox):

    host_url = trt.Unicode(
        default_value="http://sysml2-sst.intercax.com"
    )
    host_port = trt.Integer(
        default_value=9000,
        min=1,
        max=65535,
    )

    page_size = trt.Integer(
        default_value=2000,
        min=10,
    )

    _api_configuration = trt.Instance(sysml2.Configuration)
    _commits_api = trt.Instance(sysml2.CommitApi)
    _elements_api = trt.Instance(sysml2.ElementApi)
    _projects_api = trt.Instance(sysml2.ProjectApi)

    projects = trt.Dict()
    elements_by_id = trt.Dict()
    elements_by_type = trt.Dict()

    lpg = trt.Instance(SysML2LabeledPropertyGraph, args=tuple())
    rdf = trt.Instance(SysML2RDFGraph, args=tuple())

    host_url_input = trt.Instance(ipyw.Text)
    host_port_input = trt.Instance(ipyw.IntText)
    project_selector = trt.Instance(ipyw.Dropdown)
    commit_selector = trt.Instance(ipyw.Dropdown)
    download_elements = trt.Instance(ipyw.Button)
    progress_bar = trt.Instance(ipyw.IntProgress, kw=dict(
        description="Loading:",
        min=0,
        max=6,
        step=1,
        value=0,
        layout=ipyw.Layout(
            max_width = "30rem",
            visibility="hidden",
            width = "80%",
        ),
    ))
    type_selector = trt.Instance(ipyw.SelectMultiple, args=tuple())

    @trt.validate("children")
    def _validate_children(self, proposal):
        children = proposal.value
        if children:
            return children
        return [
            ipyw.HTML("<h2>SysML2 Remote Service</h2>"),
            ipyw.HBox([self.host_url_input, self.host_port_input]),
            ipyw.HBox([
                ipyw.VBox([
                    self.project_selector,
                    self.commit_selector,
                    self.progress_bar,
                ], layout=ipyw.Layout(width="80%", max_width="32rem")),
                self.download_elements,
            ]),
            ipyw.HTML("<h2>Element Types<h2>"),
            self.type_selector,
        ]

    @trt.default("_api_configuration")
    def _make_api_configuration(self):
        return sysml2.Configuration(host=self.host)

    @trt.default("_commits_api")
    def _make_commits_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.CommitApi(client)
        return api

    @trt.default("_elements_api")
    def _make_elements_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ElementApi(client)
        return api

    @trt.default("_projects_api")
    def _make_projects_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ProjectApi(client)
        return api

    @trt.default("projects")
    def _make_projects(self):
        projects = self._projects_api.get_projects()
        return {
            project.id: dict(
                created=parser.parse(
                    " ".join(project.name.split()[-6:])
                ).astimezone(timezone.utc),
                full_name=project.name,
                name=" ".join(project.name.split()[:-6]),
            )
            for project in projects
        }

    @trt.default("host_url_input")
    def _make_host_url_input(self):
        input_box = ipyw.Text(default_value=self.host_url)
        trt.link(
            (self, "host_url"),
            (input_box, "value"),
        )
        layout = input_box.layout
        layout.width = "80%"
        layout.max_width = "30rem"
        return input_box

    @trt.default("host_port_input")
    def _make_host_port_input(self):
        input_box = ipyw.IntText(
            default_value=self.host_port,
            min=1,
            max=65535,
        )
        trt.link(
            (self, "host_port"),
            (input_box, "value"),
        )
        layout = input_box.layout
        layout.width = "15%"
        layout.max_width = "6rem"
        return input_box

    @trt.default("project_selector")
    def _make_project_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Projects",
            options=self._get_project_options(),
        )
        selector.observe(self._update_commit_options, "value")
        selector.layout.width = "99%"
        selector.layout.max_width = "30rem"
        return selector

    @trt.default("commit_selector")
    def _make_commit_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Commits",
            options=self._get_commit_options(),
        )
        selector.observe(self._update_elements, "value")
        selector.layout.width = "99%"
        selector.layout.max_width = "30rem"
        return selector

    @trt.default("download_elements")
    def _make_download_elements_button(self):
        button = ipyw.Button(
            icon="cloud-download",
            tooltip="Fetch elements from remote host.",
        )
        button.on_click(self._download_elements)
        layout = button.layout
        layout.height, layout.width = "90%", "15%"
        layout.max_width = "6rem"
        return button

    @trt.observe("host_url", "host_port")
    def _update_api_configuration(self, *_):
        self._api_configuration = self._make_api_configuration()

    @trt.observe("_api_configuration")
    def _update_apis(self, *_):
        for api_type in ("commit", "element", "project"):
            api_attr = f"_{api_type}s_api"
            old_api = getattr(self, api_attr)
            api_maker = getattr(self, f"_make{api_attr}")
            setattr(self, api_attr, api_maker())
            del old_api
        self.project_selector.options = self._get_project_options()

    @property
    def host(self):
        return f"{self.host_url}:{self.host_port}"

    @property
    def selected_project_id(self):
        return self.project_selector.value

    @property
    def selected_commit_id(self):
        return self.commit_selector.value

    @property
    def elements_url(self):
        return (
            f"{self.host}/"
            f"projects/{self.selected_project_id}/"
            f"commits/{self.selected_commit_id}/"
            f"elements?page[size]={self.page_size}"
        )

    @property
    def selected_elements_by_type(self):
        element_ids = sum(map(list, self.type_selector.value), [])
        return [
            self.elements_by_id[id_]
            for id_ in element_ids
        ]

    def _update_commit_options(self, *_):
        self.commit_selector.options = self._get_commit_options()

    def _update_elements(self, *_):
        pass

    def _get_project_options(self):
        project_name_instances = Counter(
            project["name"]
            for project in self.projects.values()
        )

        return {
            data["name"] + (
                f""" ({data["created"].strftime("%Y-%m-%d %H:%M:%S")})"""
                if project_name_instances[data["name"]] > 1
                else ""
            ): id_
            for id_, data in sorted(
                self.projects.items(),
                key=lambda x: x[1]["name"],
            )
        }

    def _get_commit_options(self):
        # TODO: add more info about the commit when API provides it
        return [
            commit.id
            for commit in self._commits_api.get_commits_by_project(
                self.selected_project_id
            )
        ]

    def _download_elements(self, *_):
        progress = self.progress_bar
        progress.value = 0
        progress.layout.visibility = "visible"

        response = requests.get(self.elements_url)
        progress.value += 1
        if not response.ok:
            raise requests.HTTPError(
                f"Failed to retrieve elements from '{self.elements_url}', "
                f"reason: {response.reason}"
            )
        elements = response.json()
        progress.value += 1
        self.elements_by_id = {
            element["@id"]: element
            for element in elements
        }
        progress.value += 1
        self.elements_by_type = {
            type_: tuple([
                el["@id"]
                for el in elements
                if el["@type"] == type_
            ])
            for type_ in set(element["@type"] for element in elements)
        }
        progress.value += 1
        self.lpg._update(client=self)
        progress.value += 1
        self.rdf._update(client=self)
        progress.value += 1

        progress.value = 0
        progress.layout.visibility = "hidden"

    def by_id(id_: str):
        return self.elements_by_id[id_]

    def name_by_id(id_: str):
        return self.by_id(id_).get("Name")

    @trt.observe("elements_by_type")
    def _updated_type_selector_options(self, *_):
        self.type_selector.options = {
            f"{type_} [{len(elements)}]": elements
            for type_, elements in sorted(self.elements_by_type.items())
        }

In [ ]:
client = SysML2Client()
client

In [ ]:
client.project_selector.value = "03e3cdfe-8d90-4fc3-a414-9a188203d349"
client._download_elements()

In [ ]:
[element["name"] for element in client.selected_elements_by_type]

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types="Superclassing")
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation